<a href="https://colab.research.google.com/github/sadamzuoby/Financial-analysis-tools/blob/main/Rate_of_change_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install PyMuPDF pytesseract pdf2image tabulate
!apt-get install poppler-utils tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [51]:
from google.colab import files
uploaded = files.upload()


Saving 2021.pdf to 2021.pdf
Saving 2022.pdf to 2022.pdf
Saving 2023.pdf to 2023.pdf


In [52]:
import fitz
import pytesseract
from pdf2image import convert_from_path

def extract_text_hybrid(pdf_path):
    text = ""
    try:
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
    except:
        pass

    if len(text.strip()) < 100:
        print(f"🔁 Using OCR for: {pdf_path}")
        images = convert_from_path(pdf_path)
        for img in images:
            text += pytesseract.image_to_string(img, lang='eng+ara')

    return text


In [53]:
import re
import os

def extract_year_from_filename(filename):
    match = re.search(r"\b(20\d{2})\b", filename)
    return match.group(1) if match else filename


In [54]:
financial_items = {
    "Total Assets": "Total assets",
    "Financing – net": "Financing – net",
    "Total Equity": "Total owner’s equity – Bank’s shareholders",
    "Profit After Tax": "Total",  # inside sector section
    "Total Income": "Profit before income tax"
}


In [55]:
def extract_exact_value(text, keyword, value_position=2):
    pattern = rf"{re.escape(keyword)}[^\d\-–]*([\d٬٫.,]+(?:[^\d٬٫.,]+[\d٬٫.,]+)*)"
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        all_numbers = re.findall(r"[\d٬٫.,]+", match.group(1))
        if len(all_numbers) >= value_position:
            raw = all_numbers[value_position - 1]
            clean = raw.replace(',', '').replace('٬', '').replace('٫', '.')
            try:
                return float(clean)
            except:
                return None
    return None


In [56]:
def extract_profit_after_tax(text):
    section_pattern = r"the Bank’s business sectors.*?(?=Notes|Statement|Independent)"
    section_match = re.search(section_pattern, text, re.IGNORECASE | re.DOTALL)
    if section_match:
        section_text = section_match.group(0)
        match = re.search(r"Total[^\d]*([\d٬٫.,]+)", section_text, re.IGNORECASE)
        if match:
            raw = match.group(1).replace(',', '').replace('٬', '').replace('٫', '.')
            try:
                return float(raw)
            except:
                return None
    return None


In [58]:
financial_data = {label: {} for label in financial_items}

for filename in uploaded:
    year = extract_year_from_filename(filename)
    text = extract_text_hybrid(filename)

    for label, keyword in financial_items.items():
        if label == "Profit After Tax":
            value = extract_profit_after_tax(text)
        elif label == "Total Income":
            value = extract_exact_value(text, keyword, value_position=1)
        else:
            value = extract_exact_value(text, keyword, value_position=2)

        financial_data[label][year] = value


In [59]:
def compute_changes(values_dict):
    sorted_years = sorted(values_dict.keys())
    values = [values_dict[year] for year in sorted_years]

    changes = []
    for i in range(1, len(values)):
        old, new = values[i - 1], values[i]
        if old is not None and new is not None and old != 0:
            diff = new - old
            pct = (diff / old) * 100
            changes.append((round(diff, 2), round(pct, 2)))
        else:
            changes.append((None, None))
    return sorted_years, values, changes


In [60]:
from tabulate import tabulate

table = []

for metric, yearly_values in financial_data.items():
    years, values, changes = compute_changes(yearly_values)
    row = [metric]
    row += [f"{v:,.2f}" if v else "N/A" for v in values]
    row += [f"Δ={d:,.2f}, %{p:.2f}" if d else "N/A" for d, p in changes]
    table.append(row)

headers = ["Metric"] + years + [f"Change {years[i-1]}→{years[i]}" for i in range(1, len(years))]
print(tabulate(table, headers=headers, tablefmt="grid", numalign="right", stralign="center"))


+------------------+----------------+----------------+----------------+------------------------+------------------------+
|      Metric      |      2021      |      2022      |      2023      |    Change 2021→2022    |    Change 2022→2023    |
+==================+================+================+================+========================+========================+
|   Total Assets   |    5,953.00    |    6,190.00    |    6,423.00    |    Δ=237.00, %3.98     |    Δ=233.00, %3.76     |
+------------------+----------------+----------------+----------------+------------------------+------------------------+
| Financing – net  | 36,826,085.00  | 38,447,971.00  | 39,847,919.00  | Δ=1,621,886.00, %4.40  | Δ=1,399,948.00, %3.64  |
+------------------+----------------+----------------+----------------+------------------------+------------------------+
|   Total Equity   | 474,354,279.00 | 509,608,983.00 | 520,576,233.00 | Δ=35,254,704.00, %7.43 | Δ=10,967,250.00, %2.15 |
+------------------+----

In [61]:
for metric, yearly_values in financial_data.items():
    print(f"\n🔍 {metric}")
    for year, value in yearly_values.items():
        print(f"  {year}: {value if value is not None else '❌ Not found'}")



🔍 Total Assets
  2021: 5953.0
  2022: 6190.0
  2023: 6423.0

🔍 Financing – net
  2021: 36826085.0
  2022: 38447971.0
  2023: 39847919.0

🔍 Total Equity
  2021: 474354279.0
  2022: 509608983.0
  2023: 520576233.0

🔍 Profit After Tax
  2021: 2021.0
  2022: 2022.0
  2023: 2023.0

🔍 Total Income
  2021: 96455516.0
  2022: 95538653.0
  2023: 94552146.0
